In [9]:
import pandas as pd
import pickle
import numpy as np
import json 
from IPython.display import clear_output
import requests
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel
from pytorch_pretrained_bert.modeling import BertPreTrainedModel, BertPreTrainingHeads
from bert_utils import *
import time
from scipy import spatial

import nltk
import string,itertools
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import unicodedata
from more_itertools import locate
from functools import reduce
import pandas as pd
import re

In [10]:
contractions = json.load(open('contractions.txt','rb'))
contractions = contractions['contractions']
    
def replaceContractions(text):
    #text = text.lower()
    c_filt_text = ''
    for word in text.split(' '):
        if word in contractions:
            c_filt_text = c_filt_text+' '+contractions[word]
        else:
            c_filt_text = c_filt_text+' '+word
    return c_filt_text.strip()

def stripText(text):
    text = replaceContractions(text.lower())
    text = re.sub('(\d+[A-z]+)|(([A-z]+\d+))',' ',text) #remove alphanumeric words
    text = re.sub('-',' ', text)
    text = re.sub('\s+',' ', text)
    text = re.sub("'",' ', text)
    return text.strip()

In [11]:
#load test data
import json
import os
test_file_list = os.listdir('./test_data/')
master_text = ''

for file in test_file_list:
    curr_file = json.load(open('test_set/'+file,'rb'))
    master_text = master_text+' '+curr_file
    
sent_bucket = master_text.split('.')
sent_bucket = [ele.strip() for ele in sent_bucket if len(ele)>5]
sent_bucket = [stripText(ele) for ele in sent_bucket]

In [12]:
def getregexChunks(text, grammar):

    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))
    return [(ele[0], ele[1], ele[2], ctr) for ele,ctr in zip(all_chunks,range(len(all_chunks)))]

def getCandidatePhrases(text, pos_search_pattern_list=[r"""base: {(<JJ.*>*<NN.*>+<IN>)?<JJ>*<NN.*>+}"""]):
                                       #r"""nounverb:{<NN.+>+<.+>{0,2}<VB*>{1}}""",
                                       #r"""verbnoun:{<VB*>{1}<.+>{0,2}<NN.+>+}"""]):
                                       #r""" nounnoun:{<NN.+>+<.+>{1,2}<NN.+>+}"""]):
                                       #r"""baseverb: {(<JJ.+>+<IN>)?<JJ>*<VB.*>+}"""]):
    punct = set(string.punctuation)
    all_chunks = []

    for pattern in pos_search_pattern_list:
        all_chunks+=getregexChunks(text, pattern)
    
    candidate_locs = [' '.join(str(ctr) for word, pos, 
                           chunk,ctr in group).lower() 
                  for key, group in itertools.groupby(all_chunks, 
                  lambda_unpack(lambda word, pos, chunk, ctr: chunk != 'O')) if key]
    
    candidate_phrases = [' '.join(word for word, pos, 
                           chunk,ctr in group).lower() 
                  for key, group in itertools.groupby(all_chunks, 
                  lambda_unpack(lambda word, pos, chunk, ctr: chunk != 'O')) if key]
    
    #candidate_phrases = [cand for cand in candidates_tokens if cand not in stop_words and not all(char in punct for char in cand)]

    return candidate_phrases,candidate_locs

def lambda_unpack(f):
    return lambda args: f(*args)

def getWordLevelFeats(sent,token_feat_dict,tokenizer):
    word_feat_list = []
    for word in sent.split(' '):
        tokenized_word = tokenizer.tokenize(word)
        if len(tokenized_word)==1:
            word_feat_list.append(np.array(token_feat_dict[tokenized_word]))
        else:
            tok_feats = []
            for tok in tokenized_word:
                tok_feats+=np.array(token_feat_dict[tok])
            word_feat_list.append(tok_feats)
            
    return word_feat_list

def getWordFeatsFromBertTokenFeats(sent_tokens,bert_tokens,bert_token_feats):
    #steps for merging the bert tokens to get the BERT features for actual words
    #1. iterate over the BERT base tokenizer
    #2. lookup for the actual word in the current BERT lookup postions
    #3. If found:
        #3a. the word is not tokenized further - use the current BERT features as word embedding
    #else:
        #3b. the word is tokenized in BERT - find the sequence of tokens and sum up the features to get the word vector
    base_ctr = 0
    bert_ctr = 0
    word_feat_list = []

    for word in sent_tokens:
        if bert_tokens[bert_ctr] == word:#word not further tokenized, use the same feature vector
            word_feat_list.append(np.array(bert_token_feats[bert_ctr].detach().numpy()))
            base_ctr+=1
            bert_ctr+=1
        else:
            aggr_feats = np.array(bert_token_feats[bert_ctr].detach().numpy())
            aggr_word = bert_tokens[bert_ctr]
            merge_next = True
            while merge_next and bert_ctr<len(bert_tokens)-1:
                if '#' in bert_tokens[bert_ctr+1]:
                    aggr_word = aggr_word+bert_tokens[bert_ctr+1]
                    bert_ctr+=1
                    aggr_feats+=np.array(bert_token_feats[bert_ctr].detach().numpy())
                else:
                    merge_next = False
                    bert_ctr+=1
            word_feat_list.append(aggr_feats)
    assert len(sent_tokens)==len(word_feat_list)
    return word_feat_list

def getPOSPhrases(sent_tokens,candidate_pos_tags = ['NN','NNPS','NNS','NNP','VBG','VBN','VBP','VBZ','JJ','JJR','JJS'],
                  tag_pairs = ['NN','VB','JJ']):

    candidate_pos_tags = ['NN','NNPS','NNS','NNP','VBG','VBN','VBP','VBZ','JJ','JJR','JJS']
    tag_pairs = ['NN','VB','JJ']

    sent_tokens_pos= nltk.pos_tag(sent_tokens)
    sent_tokens = [(ele[0],ele[1],ctr) for ctr,ele in zip(range(len(sent_tokens_pos)),sent_tokens_pos) if ele[1] in candidate_pos_tags and len(ele[0])>2]

    noun_tags = [(tok,ctr) for tok,pos,ctr in sent_tokens if 'NN' in pos]
    verb_tags = [(tok,ctr) for tok,pos,ctr in sent_tokens if 'VB' in pos]
    adj_tags = [(tok,ctr) for tok,pos,ctr in sent_tokens if 'JJ' in pos]

    #return dict(zip(['noun_tags','verb_tags','adj_tags'],[[noun_tags],[verb_tags],[adj_tags]]))
    return dict(zip(['noun_tags','verb_tags','adj_tags'],[noun_tags,verb_tags,adj_tags]))

def getPOSSetsForSent(text_sent_tokens):
    pos_sent = getPOSPhrases(text_sent_tokens)
    merged_pos_list = []
    
    for key in pos_sent.keys():
        merge_list = []
        phrase_list = []

        curr_candidate_tokens = [ele[0] for ele in pos_sent[key]]
        pos_tok_seq = [ele[1] for ele in pos_sent[key]]
        
        assert len(curr_candidate_tokens) == len(pos_tok_seq)

        diff_list = [pos_tok_seq[i+1]-pos_tok_seq[i] for i in range(len(pos_tok_seq)-1)]
        diff_list.append(0)

        for ctr in range(len(pos_tok_seq)):
            if diff_list[ctr]==1:
                merge_list.append(pos_tok_seq[ctr])
                merge_list.append(pos_tok_seq[ctr+1])
            else:
                phrase_list.append(merge_list)
                merge_list = []
                if pos_tok_seq[ctr] not in reduce(lambda x,y: x+y,phrase_list):
                    phrase_list.append([pos_tok_seq[ctr]])

        merged_pos_list.append([list(set(ele)) for ele in phrase_list if len(ele)>0])
    return dict(zip(pos_sent.keys(),merged_pos_list))

def posSetFeats(text_sent_tokens,token_bert_feats):
    
    assert len(text_sent_tokens)==len(token_bert_feats)
    
    sent_pos_sets = getPOSSetsForSent(text_sent_tokens)
    pos_wise_feats = []
    pos_words = []
    for pos_key in sent_pos_sets.keys():
        curr_pos_key_feats = []
        curr_pos_entity = []
        
        for pos_set in sent_pos_sets[pos_key]:
            if len(pos_set)>1:
                feat_list = []
                sent = ''
                #merge all the tags in current list
                for token in pos_set:
                    feat_list.append(token_bert_feats[token])
                    sent = sent+' '+text_sent_tokens[token]
                curr_pos_key_feats.append(sum(feat_list))
                curr_pos_entity.append(sent.strip())
            else:
                curr_pos_key_feats.append(token_bert_feats[pos_set[0]])
                curr_pos_entity.append(text_sent_tokens[pos_set[0]])
        pos_wise_feats.append(curr_pos_key_feats)
        pos_words.append(curr_pos_entity)
        
    return dict(zip(sent_pos_sets.keys(),pos_wise_feats)),pos_words 

###################### Key-phrase based cosine similarity ######################
def getKeyPhraseFeatures(kp_list, kp_loc_idx,text_feats, text_tokens):
    
    key_phrase_feats = []
    for ele,loc_list in zip(kp_list,kp_loc_idx):
        if len(ele.split(' '))==1:
            idx_val = int(loc_list[0])
            key_phrase_feats.append(getTokenFeature(ele,idx_val,text_feats,text_tokens))
        else:
            curr_feature_vec = []
            for tok,tok_idx in zip(ele.split(' '),loc_list.split(' ')):
                curr_feature_vec.append(getTokenFeature(tok,int(tok_idx),text_feats,text_tokens))
            key_phrase_feats.append(sum(curr_feature_vec))
    return key_phrase_feats
            
def getTokenFeature(token, token_idx, text_feats, text_tokens):    
    if text_tokens[token_idx]==token:
        feat_vec = text_feats[token_idx]
    else:
        #print('Token not found in the location, searching entire text.: ', token)
        if token in text_tokens:
            idx_val = text_tokens.index(token)
            feat_vec = text_feats[idx_val]
        else:
            #print('Token not found.. returning default feature vector: ', token)
            feat_vec = np.full(len(text_feats[0]),0.01)
    return feat_vec

## ----------------- Methods borrowed from BERT tokenizer -----------------

def tokenize(text, never_split = [], do_lower_case = True):
    """Tokenizes a piece of text."""
    text = _clean_text(text)
    # This was added on November 1st, 2018 for the multilingual and Chinese
    # models. This is also applied to the English models now, but it doesn't
    # matter since the English models were not trained on any Chinese data
    # and generally don't have any Chinese data in them (there are Chinese
    # characters in the vocabulary because Wikipedia does have some Chinese
    # words in the English Wikipedia.).
    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
        if do_lower_case and token not in never_split:
            token = token.lower()
            token = _run_strip_accents(token)
        split_tokens.extend(_run_split_on_punc(token))

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    return output_tokens


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

def _run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1

    return ["".join(x) for x in output]

def _clean_text(text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
        cp = ord(char)
        if cp == 0 or cp == 0xfffd or _is_control(char):
            continue
        if _is_whitespace(char):
            output.append(" ")
        else:
            output.append(char)
    return "".join(output)

def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False

def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False

def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

def _run_strip_accents(text):
    """Strips accents from a piece of text."""
    text = unicodedata.normalize("NFD", text)
    output = []
    for char in text:
        cat = unicodedata.category(char)
        if cat == "Mn":
            continue
        output.append(char)
    return "".join(output)

def getKPBasedSimilarity(text1, text2,  bert_layer = -1):
    
    
    print(text1)
    print()
    print(text2)
    print()
    
    token_feats_1,final_feats1,text1_bert_tokenized = getBERTFeatures(model, text1, attn_head_idx=layer)
    token_feats_2,final_feats2,text2_bert_tokenized = getBERTFeatures(model, text2, attn_head_idx=layer)

    text1_sent_tokens = tokenize(text1)
    text2_sent_tokens = tokenize(text2)

    merged_feats_text1 = getWordFeatsFromBertTokenFeats(text1_sent_tokens,text1_bert_tokenized,token_feats_1)
    merged_feats_text2 = getWordFeatsFromBertTokenFeats(text2_sent_tokens,text2_bert_tokenized,token_feats_2)

    #get candidate key-phrases for both sentences
    kps_sent1,kps_loc_sent1 = getCandidatePhrases(text1)
    kps_sent2,kps_loc_sent2 = getCandidatePhrases(text2)

    print(kps_sent1)
    print()
    print(kps_sent2)
    print()
    
    sent1_kp_feats = getKeyPhraseFeatures(kps_sent1,kps_loc_sent1,merged_feats_text1,text1_sent_tokens)
    sent2_kp_feats = getKeyPhraseFeatures(kps_sent2,kps_loc_sent2,merged_feats_text2,text2_sent_tokens)

    curr_max = 0
    for sent1_kp, feats1 in zip(kps_sent1,sent1_kp_feats):
        for sent2_kp, feats2 in zip(kps_sent2,sent2_kp_feats):
            if len(sent1_kp)<3 or len(sent2_kp)<3:
                curr_sim = 0.1
            else:
                curr_sim = 1-spatial.distance.cosine(feats1,feats2)
            print(sent1_kp,'<>',sent2_kp,': ',curr_sim)
            if curr_sim>curr_max:
                curr_max = curr_sim

#     print()      
#     print('-----------------------------------------------------------')
#     print('final similarity:', curr_max)
#     print('-----------------------------------------------------------')
#     print("Sentence level score: ", 1-spatial.distance.cosine(final_feats1,final_feats2))
#     print('-----------------------------------------------------------')
    
    return curr_max

def getPOSBasedSimilarity(text1, text2,  bert_layer = -1):

    token_feats_1,final_feats1,text1_bert_tokenized = getBERTFeatures(model, text1, attn_head_idx=layer)
    token_feats_2,final_feats2,text2_bert_tokenized = getBERTFeatures(model, text2, attn_head_idx=layer)

    text1_sent_tokens = tokenize(text1)
    text2_sent_tokens = tokenize(text2)

    merged_feats_text1 = getWordFeatsFromBertTokenFeats(text1_sent_tokens,text1_bert_tokenized,token_feats_1)
    merged_feats_text2 = getWordFeatsFromBertTokenFeats(text2_sent_tokens,text2_bert_tokenized,token_feats_2)

    sent1_pos_feats,sent1_pos = posSetFeats(text1_sent_tokens,merged_feats_text1)
    sent2_pos_feats,sent2_pos = posSetFeats(text2_sent_tokens,merged_feats_text2)

    #Do pos-tag wise feature similarity and take max() similarity in each pos as metric

    #get pos_tag wise cosine similarity
    pos_idx = 0 #0-noun, 1-verb, 2 - adj
    curr_pos = list(sent1_pos_feats.keys())[pos_idx]
    sent1_tags = sent1_pos[pos_idx]
    sent2_tags = sent2_pos[pos_idx]

    #print('Current POS: ',curr_pos)
    #print()
    sent1_curr_pos_feats = sent1_pos_feats[curr_pos]
    sent2_curr_pos_feats = sent2_pos_feats[curr_pos]

    curr_max = 0
    if len(sent1_curr_pos_feats)>0 and len(sent2_curr_pos_feats)>0:
        for sent1_ctr in range(len(sent1_curr_pos_feats)):
            for sent2_ctr in range(len(sent2_curr_pos_feats)):
                curr_sim = 1-spatial.distance.cosine(sent1_curr_pos_feats[sent1_ctr],sent2_curr_pos_feats[sent2_ctr])
                if curr_sim>curr_max:
                    curr_max = curr_sim
                #print(sent1_tags[sent1_ctr],'<>',sent2_tags[sent2_ctr],":",dist)
            #print()
#     else:
#         print("No %s tokens in one of the texts, skipping" % (curr_pos))
#     print('-----------------------------------------------------------')
#     print('final similarity:', curr_max)
#     print('-----------------------------------------------------------')
#     print("Sentence level score: ", 1-spatial.distance.cosine(final_feats1,final_feats2))
#     print('-----------------------------------------------------------')

    return curr_max

def getCosineSimilarity(text1, text2,  bert_layer = -1):

    token_feats_1,final_feats1,text1_bert_tokenized = getBERTFeatures(model, text1, attn_head_idx=layer)
    token_feats_2,final_feats2,text2_bert_tokenized = getBERTFeatures(model, text2, attn_head_idx=layer)

    return 1-spatial.distance.cosine(final_feats1,final_feats2)

In [13]:
def getKPBasedSimilarity_loop(tup1,tup2, text1, text2, bert_layer = -1):
    
    token_feats_1,final_feats1,text1_bert_tokenized = tup1
    token_feats_2,final_feats2,text2_bert_tokenized = tup2

    text1_sent_tokens = tokenize(text1)
    text2_sent_tokens = tokenize(text2)

    merged_feats_text1 = getWordFeatsFromBertTokenFeats(text1_sent_tokens,text1_bert_tokenized,token_feats_1)
    merged_feats_text2 = getWordFeatsFromBertTokenFeats(text2_sent_tokens,text2_bert_tokenized,token_feats_2)

    #get candidate key-phrases for both sentences
    kps_sent1,kps_loc_sent1 = getCandidatePhrases(text1)
    kps_sent2,kps_loc_sent2 = getCandidatePhrases(text2)

    sent1_kp_feats = getKeyPhraseFeatures(kps_sent1,kps_loc_sent1,merged_feats_text1,text1_sent_tokens)
    sent2_kp_feats = getKeyPhraseFeatures(kps_sent2,kps_loc_sent2,merged_feats_text2,text2_sent_tokens)

    curr_max = 0
    for sent1_kp, feats1 in zip(kps_sent1,sent1_kp_feats):
        for sent2_kp, feats2 in zip(kps_sent2,sent2_kp_feats):
            if len(sent1_kp)<3 or len(sent2_kp)<3:
                curr_sim = 0.1
            else:
                curr_sim = 1-spatial.distance.cosine(feats1,feats2)
            print(sent1_kp,'<>',sent2_kp,': ',curr_sim)
            if curr_sim>curr_max:
                curr_max = curr_sim


#     print()      
#     print('-----------------------------------------------------------')
#     print('final similarity:', curr_max)
#     print('-----------------------------------------------------------')
#     print("Sentence level score: ", 1-spatial.distance.cosine(final_feats1,final_feats2))
#     print('-----------------------------------------------------------')
    
    return curr_max

def getPOSBasedSimilarity_loop(tup1, tup2,  text1, text2, bert_layer = -1):

    token_feats_1,final_feats1,text1_bert_tokenized = tup1
    token_feats_2,final_feats2,text2_bert_tokenized = tup2

    text1_sent_tokens = tokenize(text1)
    text2_sent_tokens = tokenize(text2)

    merged_feats_text1 = getWordFeatsFromBertTokenFeats(text1_sent_tokens,text1_bert_tokenized,token_feats_1)
    merged_feats_text2 = getWordFeatsFromBertTokenFeats(text2_sent_tokens,text2_bert_tokenized,token_feats_2)

    sent1_pos_feats,sent1_pos = posSetFeats(text1_sent_tokens,merged_feats_text1)
    sent2_pos_feats,sent2_pos = posSetFeats(text2_sent_tokens,merged_feats_text2)

    #Do pos-tag wise feature similarity and take max() similarity in each pos as metric

    #get pos_tag wise cosine similarity
    pos_idx = 0 #0-noun, 1-verb, 2 - adj
    curr_pos = list(sent1_pos_feats.keys())[pos_idx]
    sent1_tags = sent1_pos[pos_idx]
    sent2_tags = sent2_pos[pos_idx]

    #print('Current POS: ',curr_pos)
    #print()
    sent1_curr_pos_feats = sent1_pos_feats[curr_pos]
    sent2_curr_pos_feats = sent2_pos_feats[curr_pos]

    curr_max = 0
    if len(sent1_curr_pos_feats)>0 and len(sent2_curr_pos_feats)>0:
        for sent1_ctr in range(len(sent1_curr_pos_feats)):
            for sent2_ctr in range(len(sent2_curr_pos_feats)):
                dist = 1-spatial.distance.cosine(sent1_curr_pos_feats[sent1_ctr],sent2_curr_pos_feats[sent2_ctr])
                if dist>curr_max:
                    curr_max = dist
                #print(sent1_tags[sent1_ctr],'<>',sent2_tags[sent2_ctr],":",dist)
            #print()
#     else:
#         print("No %s tokens in one of the texts, skipping" % (curr_pos))
#     print('-----------------------------------------------------------')
#     print('final similarity:', curr_max)
#     print('-----------------------------------------------------------')
#     print("Sentence level score: ", 1-spatial.distance.cosine(final_feats1,final_feats2))
#     print('-----------------------------------------------------------')

    return curr_max

def getCosineSimilarity_loop(tup1, tup2,  bert_layer = -1):

    token_feats_1,final_feats1,text1_bert_tokenized = tup1
    token_feats_2,final_feats2,text2_bert_tokenized = tup2

    return 1-spatial.distance.cosine(final_feats1,final_feats2)

In [14]:
# experiment_id, experiment_name
# 01daapwr6w051q9wwqy99jsgfy - Generic
# 01daaqy88qzb19jqz5prjfr76y - Engineering
# 01daaqyn9gbebc92aywnxedp0c - HR
# 01daatanxnrqa35e6004hb7mbn - Marketing
# 01daatbc3ak1qwc5nyc5ahv2xz - Product
# 01dadp74wfv607knpcb6vvxgtg - AI
# 01daayheky5f4e02qvrjptftxv - Ether Engineering

In [15]:
config_path = '/Users/venkat/Documents/mlflow/mlflow_bert/bert_mlflow_pyfunc/artifacts/bert_config.json'
mind_path = '/Users/venkat/Documents/mlflow/mlflow_bert/bert_mlflow_pyfunc/artifacts/mind.pkl'
model_path = '/Users/venkat/Documents/mlflow/mlflow_bert_deploy/mind-01daaqyn9gbebc92aywnxedp0c/artifacts/model.bin'

config = BertConfig.from_json_file(config_path)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
mind_dict = pickle.load(open(mind_path,'rb'))

model = BertForPreTrainingCustom(config)
state_dict = torch.load(model_path,map_location='cpu')

model.load_state_dict(state_dict)
model.eval()
print()

In [16]:
#calculate pair-wise cosine similarity
layer = -1

text1_list = []
text2_list = []
#pos_dist_list = []
kp_dist_list = []
cs_dist_list = []

sent_feat_list = []
for text in sent_bucket:
    sent_feat_list.append(getBERTFeatures(model, text, attn_head_idx=layer))
    
for i in range(len(sent_bucket)):
    sent1 = sent_bucket[i]
    for j in range(i+1,len(sent_bucket)):
        sent2 = sent_bucket[j]
        text1_list.append(sent1)
        text2_list.append(sent2)
        tup1 = sent_feat_list[i]
        tup2 = sent_feat_list[j]
        #pos_dist_list.append(getPOSBasedSimilarity_loop(tup1, tup2, sent1, sent2))
        kp_dist_list.append(getKPBasedSimilarity_loop(tup1, tup2,sent1, sent2))
        cs_dist_list.append(getCosineSimilarity_loop(tup1, tup2))

print()

simple network with terraform <> lot :  0.7033495306968689
simple network with terraform <> terraform :  0.8302325010299683
simple network with terraform <> compute instances :  0.7817032337188721
simple network with terraform <> firewall rules :  0.7962830662727356
gcp <> lot :  0.635649561882019
gcp <> terraform :  0.7179625034332275
gcp <> compute instances :  0.7085735201835632
gcp <> firewall rules :  0.7305143475532532
simple network with terraform <> next thing :  0.7364122271537781
simple network with terraform <> necessary credentials from gcp :  0.7998179793357849
gcp <> next thing :  0.6469058990478516
gcp <> necessary credentials from gcp :  0.7542247176170349
simple network with terraform <> gcp console menu :  0.7369287014007568
simple network with terraform <> select apis :  0.7202449440956116
simple network with terraform <> services :  0.6676577925682068
simple network with terraform <> credentials :  0.685900866985321
gcp <> gcp console menu :  0.7173702716827393
gcp 

simple network with terraform <> file :  0.5824180841445923
simple network with terraform <> sensitive information :  0.4985794126987457
simple network with terraform <> public repository :  0.5510944128036499
gcp <> file :  0.5280028581619263
gcp <> sensitive information :  0.45927754044532776
gcp <> public repository :  0.526122510433197
simple network with terraform <> hr strategies :  0.7031657099723816
simple network with terraform <> strategic recruiting :  0.7398914694786072
simple network with terraform <> retention processes :  0.7103188037872314
simple network with terraform <> organization :  0.6984660029411316
gcp <> hr strategies :  0.6309728026390076
gcp <> strategic recruiting :  0.6515409350395203
gcp <> retention processes :  0.6411375999450684
gcp <> organization :  0.6101254224777222
simple network with terraform <> clear need for increased visibility :  0.7060657143592834
simple network with terraform <> sales :  0.6747782230377197
gcp <> clear need for increased vi

compute instances <> terraform code :  0.7336243987083435
firewall rules <> json file :  0.7371116876602173
firewall rules <> computer :  0.701958417892456
firewall rules <> file :  0.6925769448280334
firewall rules <> folder :  0.6034924387931824
firewall rules <> terraform code :  0.7288258671760559
lot <> file :  0.5984790921211243
lot <> sensitive information :  0.5363341569900513
lot <> public repository :  0.5612996220588684
terraform <> file :  0.5832033753395081
terraform <> sensitive information :  0.5017414689064026
terraform <> public repository :  0.5212568640708923
compute instances <> file :  0.5822255611419678
compute instances <> sensitive information :  0.5304741263389587
compute instances <> public repository :  0.5800338387489319
firewall rules <> file :  0.6032566428184509
firewall rules <> sensitive information :  0.5490624904632568
firewall rules <> public repository :  0.6040897965431213
lot <> i :  0.1
lot <> kinds of apps i :  0.6552579402923584
terraform <> i 

terraform <> select apis :  0.6965315937995911
terraform <> services :  0.6503278613090515
terraform <> credentials :  0.6868176460266113
compute instances <> gcp console menu :  0.7211131453514099
compute instances <> select apis :  0.743038535118103
compute instances <> services :  0.7033607363700867
compute instances <> credentials :  0.7035790085792542
firewall rules <> gcp console menu :  0.7305580377578735
firewall rules <> select apis :  0.7604591250419617
firewall rules <> services :  0.7055060267448425
firewall rules <> credentials :  0.7009241580963135
lot <> create credentials button :  0.6574373841285706
lot <> choose service account key :  0.6262028217315674
terraform <> create credentials button :  0.7077409029006958
terraform <> choose service account key :  0.7077256441116333
compute instances <> create credentials button :  0.7422086000442505
compute instances <> choose service account key :  0.7294735312461853
firewall rules <> create credentials button :  0.744856655

compute instances <> today :  0.5297000408172607
compute instances <> much vying :  0.5474939942359924
compute instances <> attention from social media :  0.6163706183433533
compute instances <> engine optimization :  0.5586349368095398
compute instances <> click advertising :  0.5618466734886169
compute instances <> hair :  0.5736733675003052
firewall rules <> message :  0.5994914770126343
firewall rules <> right audience :  0.5830695033073425
firewall rules <> visibility :  0.6152640581130981
firewall rules <> sales :  0.6108180284500122
firewall rules <> profit :  0.5643664002418518
firewall rules <> offer :  0.5491023659706116
firewall rules <> today :  0.5477012395858765
firewall rules <> much vying :  0.5619986057281494
firewall rules <> attention from social media :  0.6223071217536926
firewall rules <> engine optimization :  0.6039102673530579
firewall rules <> click advertising :  0.6043628454208374
firewall rules <> hair :  0.5903322696685791
lot <> truth :  0.637623131275177

...

public repository <> hair :  0.43058109283447266
file <> truth :  0.5750097036361694
file <> point in business :  0.5461763739585876
file <> next level :  0.5255388021469116
sensitive information <> truth :  0.5361959934234619
sensitive information <> point in business :  0.5264109373092651
sensitive information <> next level :  0.5157243013381958
public repository <> truth :  0.5117297172546387
public repository <> point in business :  0.5653414726257324
public repository <> next level :  0.5698559880256653
file <> fray :  0.5237378478050232
sensitive information <> fray :  0.47511160373687744
public repository <> fray :  0.5269913077354431
file <> entrepreneurs :  0.5068826675415039
file <> businesses :  0.5779948830604553
file <> businesses :  0.5779948830604553
sensitive information <> entrepreneurs :  0.4167223274707794
sensitive information <> businesses :  0.5563185214996338
sensitive information <> businesses :  0.5563185214996338
public repository <> entrepreneurs :  0.4079476

hr strategies <> result :  0.508587658405304
hr strategies <> day :  0.5438027381896973
hr strategies <> day operations :  0.6170976161956787
hr strategies <> company :  0.6072874665260315
hr strategies <> customer hand :  0.5778586864471436
hr strategies <> supply chain demands :  0.606802761554718
hr strategies <> right marketing strategies :  0.7008188962936401
hr strategies <> business s growth :  0.63453209400177
strategic recruiting <> result :  0.4966619610786438
strategic recruiting <> day :  0.5892693996429443
strategic recruiting <> day operations :  0.675225555896759
strategic recruiting <> company :  0.6694546341896057
strategic recruiting <> customer hand :  0.6933043599128723
strategic recruiting <> supply chain demands :  0.706113874912262
strategic recruiting <> right marketing strategies :  0.7719807624816895
strategic recruiting <> business s growth :  0.6917837262153625
retention processes <> result :  0.5227707624435425
retention processes <> day :  0.58553260564804

order <> truth :  0.5536348819732666
order <> point in business :  0.6310405731201172
order <> next level :  0.6227957606315613
visibility <> truth :  0.6858922839164734
visibility <> point in business :  0.6917903423309326
visibility <> next level :  0.6695544123649597
businesses <> truth :  0.6555250287055969
businesses <> point in business :  0.7038989663124084
businesses <> next level :  0.6687332391738892
money <> truth :  0.6158102750778198
money <> point in business :  0.6924826502799988
money <> next level :  0.6847992539405823
order <> fray :  0.4884242117404938
visibility <> fray :  0.6564586162567139
businesses <> fray :  0.62158203125
money <> fray :  0.5712242126464844
order <> entrepreneurs :  0.4456438720226288
order <> businesses :  0.6076720356941223
order <> businesses :  0.6076720356941223
visibility <> entrepreneurs :  0.6782413125038147
visibility <> businesses :  0.7484115958213806
visibility <> businesses :  0.7484115958213806
businesses <> entrepreneurs :  0.668

businesses <> fray :  0.5707688331604004
backs of social media <> fray :  0.6161086559295654
businesses <> entrepreneurs :  0.8263787031173706
businesses <> businesses :  0.712884247303009
businesses <> businesses :  0.712884247303009
backs of social media <> entrepreneurs :  0.6659939289093018
backs of social media <> businesses :  0.6729409098625183
backs of social media <> businesses :  0.6729409098625183
businesses <> result :  0.38400381803512573
businesses <> day :  0.39625823497772217
businesses <> day operations :  0.47274941205978394
businesses <> company :  0.50888991355896
businesses <> customer hand :  0.47540947794914246
businesses <> supply chain demands :  0.4736862778663635
businesses <> right marketing strategies :  0.508621096611023
businesses <> business s growth :  0.5138648748397827
backs of social media <> result :  0.46309134364128113
backs of social media <> day :  0.476673424243927
backs of social media <> day operations :  0.53926020860672
backs of social medi

hair <> day operations :  0.5673902630805969
hair <> company :  0.6100374460220337
hair <> customer hand :  0.6192629933357239
hair <> supply chain demands :  0.6093231439590454
hair <> right marketing strategies :  0.6344782114028931
hair <> business s growth :  0.6841169595718384
truth <> fray :  0.5897005796432495
point in business <> fray :  0.6186079382896423
next level <> fray :  0.6092283725738525
truth <> entrepreneurs :  0.6059399843215942
truth <> businesses :  0.6558205485343933
truth <> businesses :  0.6558205485343933
point in business <> entrepreneurs :  0.5488950610160828
point in business <> businesses :  0.7040399312973022
point in business <> businesses :  0.7040399312973022
next level <> entrepreneurs :  0.5297229290008545
next level <> businesses :  0.6597060561180115
next level <> businesses :  0.6597060561180115
truth <> result :  0.5212628841400146
truth <> day :  0.4799610674381256
truth <> day operations :  0.5263763666152954
truth <> company :  0.5333083868026

In [17]:
score_df = pd.DataFrame({'Sent1': text1_list,
                        'Sent2': text2_list,
                        'KP_Similarity': kp_dist_list,
                        'Cosine_Similarity': cs_dist_list})
score_df = score_df[score_df['Sent1']!=score_df['Sent2']]
print(score_df.shape)
#score_df = score_df[score_df['KP_Similarity']<1]

(621, 4)


In [18]:
score_df['cs_kp_diff'] = score_df['Cosine_Similarity']-score_df['KP_Similarity']
score_df = score_df[score_df['KP_Similarity']>0]
score_df.sort_values(by='cs_kp_diff', ascending=False)

,Sent1,Sent2,KP_Similarity,Cosine_Similarity,cs_kp_diff
472,"and with that in mind, i tried many approaches...",how do you get your message to the right audie...,0.649831,0.823917,0.174086
325,"if you work in the javascript ecosystem, you a...",how do you get your message to the right audie...,0.661606,0.822899,0.161294
623,how do you get your message to the right audie...,as a result of dealing with the day to day ope...,0.711773,0.870922,0.159149
300,"and with that in mind, i tried many approaches...",how do you get your message to the right audie...,0.625579,0.768445,0.142866
64,you can do a lot more with terraform such as s...,how do you get your message to the right audie...,0.622307,0.762091,0.139784
435,you can do a lot more with terraform such as s...,how do you get your message to the right audie...,0.622307,0.762091,0.139784
394,maintaining an application with a large codeba...,how do you get your message to the right audie...,0.671347,0.806298,0.134951
547,this will download a json file to your compute...,how do you get your message to the right audie...,0.609668,0.743621,0.133953
219,this will download a json file to your compute...,how do you get your message to the right audie...,0.609668,0.743621,0.133953
351,when one of your core libraries releases a maj...,"if you are feeling stuck, join the fray",0.645090,0.777972,0.132883


In [19]:
idx = 472
text1 = score_df['Sent1'][idx]
text2 = score_df['Sent2'][idx]
getKPBasedSimilarity(text1,text2)

and with that in mind, i tried many approaches and frameworks for implementing the same pattern: single page applications (spa)

how do you get your message to the right audience and do it effectively? how do you boost visibility and increase sales while sustaining a profit with a converting offer? today, with so much vying for our attention from social media, to search engine optimization, blogging and pay per click advertising, it is easy to see why most are ready to pull their hair out

['mind', 'many approaches', 'frameworks', 'same pattern', 'single page applications', 'spa']

['message', 'right audience', 'visibility', 'sales', 'profit', 'offer', 'today', 'much vying', 'attention from social media', 'engine optimization', 'click advertising', 'hair']

mind <> message :  0.5401946306228638
mind <> right audience :  0.5094702243804932
mind <> visibility :  0.5330876111984253
mind <> sales :  0.5251703858375549
mind <> profit :  0.528603732585907
mind <> offer :  0.4492456614971161


0.6498307585716248

In [ ]:
replaceContractions("I didn't do that")